Модифіковуємо зображення виділяючи контури


In [2]:
import cv2
import os

# src_dir = './RUSSIAN_PLANES_DB/train'
# src_dir = '../RUSSIAN_PLANAE_DB/test_10/'
# src_dir = '../RUSSIAN_PLANAE_DB/original/'
# dest_dir = '../RUSSIAN_PLANAE_DB/full_modified/'

src_dir =  "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\original_src"
dest_dir = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"


for filename in os.listdir(src_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        img_path = os.path.join(src_dir, filename)
        img = cv2.imread(img_path)
        print(img)
        cv2.imshow('image', img)
        print(os.path.exists(img_path))
        #CANNY
        # edges = cv2.Canny(img,150,250)

        #LAPLACE
        # LESS_CONTRAST_EDGES (ver 1)
        # denoise = cv2.GaussianBlur(img, (3,3), 0)
        # MORE_CONTRAST_EDGES (ver 2)
        denoise = cv2.GaussianBlur(img, (1,1), 0)
        
        #FURTHER_PROCESSING
        to_gray = cv2.cvtColor(denoise,  cv2.COLOR_BGR2GRAY)
        res_lap = cv2.Laplacian(to_gray, cv2.CV_16S, ksize=3)

        cv2.imwrite(os.path.join(dest_dir, filename), res_lap)
        # cv2.imwrite(os.path.join('./test_10_images/', filename), denoise) # last arg denoise or edges
        # cv2.imwrite(os.path.join('./test_10_images/', filename), to_gray)


[[[100 100 118]
  [ 98  98 116]
  [ 94  94 112]
  ...
  [ 75  86  67]
  [ 73  84  64]
  [ 74  85  65]]

 [[ 99  99 117]
  [ 97  97 115]
  [ 95  95 113]
  ...
  [ 71  86  65]
  [ 72  87  64]
  [ 76  91  67]]

 [[ 96  96 114]
  [ 95  95 113]
  [ 95  95 113]
  ...
  [ 72  90  67]
  [ 75  92  68]
  [ 81  98  71]]

 ...

 [[101 110 119]
  [103 112 121]
  [108 115 126]
  ...
  [109 108 102]
  [ 99  97  91]
  [ 94  92  87]]

 [[104 113 124]
  [107 115 127]
  [116 123 137]
  ...
  [109 109 106]
  [ 99 101  95]
  [ 95  96  91]]

 [[109 117 130]
  [116 123 137]
  [123 129 146]
  ...
  [114 115 109]
  [104 106 100]
  [ 96  97  93]]]
True
[[[ 93  95 113]
  [ 92  94 112]
  [ 87  89 107]
  ...
  [ 94  97 104]
  [ 93  96 104]
  [ 92  95 101]]

 [[ 93  95 113]
  [ 90  92 110]
  [ 86  88 106]
  ...
  [ 93  96 103]
  [ 93  96 104]
  [ 93  96 102]]

 [[ 91  93 111]
  [ 89  91 109]
  [ 93  95 113]
  ...
  [ 91  95 101]
  [ 91  94 102]
  [ 92  96 102]]

 ...

 [[130 137 154]
  [143 150 165]
  [154 161 176]

KeyboardInterrupt: 

Для розподілення датасету по train val папках


In [ ]:
import os
import cv2
import shutil


# Шлях до папки з зображеннями
src_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"
train_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\train"
val_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\val"

# Отримати список всіх файлів у початковій папці
all_files = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]

# Відсортувати файли (опціонально, залежить від вимог)
all_files.sort()

# Останні 50 файлів для валідаційного набору
val_files = all_files[-50:]
# Решта файлів для тренувального набору
train_files = all_files[:-50]

# Створити папки train та val, якщо вони ще не існують
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Переміщення файлів
for file_name in train_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(train_folder, file_name))

for file_name in val_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(val_folder, file_name))

print("Файли успішно розподілені між train та val папками.")


Тренування моделі

In [1]:
# TRAINING & TEST MODEL
import torch
from ultralytics import YOLO
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'avialable device: {device}')
torch.backends.cudnn.enabled = False

# Load a model
# project_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova"
# model = YOLO('yolov8n.yaml')
# data = project_folder + 'src/CustomPlane_copy.yaml'

# load model
data = "CustomPlane.yaml"
model = YOLO('yolov8n.yaml')
model = YOLO('pts\\yolov8n.pt')

# Train and use the model with 527 images
results = model.train(data=data, epochs=200, imgsz=720, augment=True, device=0)
results = model.val()
results = model('..\\DB\\images\\train\\0266.png')

avialable device: cuda
New https://pypi.org/project/ultralytics/8.2.19 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.18  Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=pts\yolov8n.pt, data=CustomPlane.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=720, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed

train: Scanning C:\Users\Северин\Documents\unik\kursova\DB\labels\train.cache... 469 images, 2 backgrounds, 0 corrupt: 100%|██████████| 471/471 [00:00<?, ?it/s]
val: Scanning C:\Users\Северин\Documents\unik\kursova\DB\labels\val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to c:\Users\\Documents\unik\kursova\runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00037, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 736 train, 736 val
Using 8 dataloader workers
Logging results to c:\Users\\Documents\unik\kursova\runs\detect\train2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      5.47G      3.038      5.548      1.414        183        736: 100%|██████████| 30/30 [01:06<00:00,  2.22s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]

                   all         50        756          0          0          0          0


...


200 epochs completed in 3.721 hours.
Optimizer stripped from c:\Users\\Documents\unik\kursova\runs\detect\train2\weights\last.pt, 6.3MB
Optimizer stripped from c:\Users\\Documents\unik\kursova\runs\detect\train2\weights\best.pt, 6.3MB

Validating c:\Users\\Documents\unik\kursova\runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.2.18  Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3010133 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.09s/it]


                   all         50        756      0.254      0.197      0.221      0.126
            Helicopter         50         44     0.0935     0.0227     0.0632     0.0224
USSR|CIV/Transport|AN-124         50         33      0.475      0.247      0.335      0.201
USSR|CIV/Transport/Recon.|AN-24/26/30         50         21      0.181      0.286      0.176      0.102
  USSR|Transport|AN-12         50          6     0.0203      0.167     0.0178    0.00989
USSR|Transport/Refuel.|IL-76/78         50        121       0.54      0.545      0.559      0.318
USSR|Surv./MPA|IL-20/38         50          6          0          0      0.088     0.0276
       USSR|AWACS|A-50         50          1          1          0      0.332      0.298
USSR|Strat.Bomber|TU-22M         50         82      0.273      0.427      0.304      0.147
USSR|Strat.Bomber/MAP|TU-95/142         50         74      0.548      0.376      0.532      0.358
          CIV AIRCRAFT         50         34      0.385      0.676     

val: Scanning C:\Users\Северин\Documents\unik\kursova\DB\labels\val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


                   all         50        756      0.255      0.203       0.26      0.163
            Helicopter         50         44     0.0773     0.0227     0.0705     0.0267
USSR|CIV/Transport|AN-124         50         33       0.42      0.241      0.308      0.181
USSR|CIV/Transport/Recon.|AN-24/26/30         50         21      0.223      0.333      0.189      0.107
  USSR|Transport|AN-12         50          6     0.0203      0.167     0.0193     0.0103
USSR|Transport/Refuel.|IL-76/78         50        121      0.521      0.504      0.547      0.308
USSR|Surv./MPA|IL-20/38         50          6          0          0     0.0591     0.0169
       USSR|AWACS|A-50         50          1          1          0      0.995      0.895
USSR|Strat.Bomber|TU-22M         50         82       0.27      0.415      0.281      0.143
USSR|Strat.Bomber/MAP|TU-95/142         50         74      0.583      0.392      0.538       0.37
          CIV AIRCRAFT         50         34      0.397      0.735     

In [1]:
test = model('..\\DB\\images\\train\\0266.png')
cv2.imshow("img", test)

NameError: name 'model' is not defined

In [ ]:
#IMAGE_PROCESSING
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')
model = YOLO('CustomPlane_copy.yaml')
model = YOLO('./pts/yolov8n.pt')

hyp = {
       'flipud': 0.5,
       'fliplr': 0.5,
       'rotate': 0.5,
       'mosaic': 0.3,
       'mixup':  0.05
}

# Train and use the model with 527 images
results = model.train(data='./CustomPlane_copy.yaml', epochs=500, imgsz=1000, hyp=hyp) 
results = model.val()
# results = model('../RUSSIAN_PLANAE_DB/original/0266.png')



In [ ]:
import cv2
# model = YOLO('D:/DesktopFolders/Універ/5курс/Сем_2/ООП/neuron/ultralytics-main/examples/best_laplace_contrast.pt')
model = YOLO('./pts/best_laplace_contrast.pt')
# image='D:/temp_2/0223.png'
image='../img_db/RUSSIAN_PLANAE_DB/0223.png'
img = cv2.imread(image)
class_name_dict = {
    0: 'Helicopter',
    1:'USSR|CIV/Transport|AN-124',
    2:'USSR|CIV/Transport/Recon.|AN-24/26/30',
    3:'USSR|Transport|AN-22',
    4:'USSR|Transport|AN-12',
    5:'USSR|CIV/Transport|AN-2',
    6:'USSR|Transport/Refuel.|IL-76/78',
    7:'USSR|Surv./MPA|IL-20/38',
    8:'CZECHOSLOVAKIA|Trainer/G-A|L-39',
    9:'RUSSIA|Trainer|YAK-130',
    10:'USSR|AWACS|A-50',
    11:'USSR|Bomber|TU-22',
    12:'USSR|Strat.Bomber|TU-22M',
    13:'USSR|Strat.Bomber/MAP|TU-95/142',
    14:'USSR|Strat.Bomber|TU-160',
    15:'USSR|Trainer|TU-134',
    16:'CIV AIRCRAFT',
    17:'USSR|Interceptor/G-A|MIG-31',
    18:'USSR|Fighter|MIG-29',
    19:'USSR|Mutlirole Fighter|SU-27/30/33/35',
    20:'USSR|Fighter/Bomber|SU-34',
    21:'USSR|CAS|SU-25',
    22:'USSR|Tact.Bomber|SU-24'
}
results = model(img)[0]   
for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(img, class_name_dict[int(class_id)].upper(), (int(x1), int(y1 - 10)),
            cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.imwrite('recognized.jpg',img)

In [ ]:
# colab version of train


from ultralytics import YOLO
import os

# Load a model
# model = YOLO('yolov8n.yaml')
data = project_folder + 'src/CustomPlane_copy.yaml'

# load model
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

# Train and use the model with 527 images
results = model.train(data=data, epochs=100, imgsz=720, augment=True, batch=16)
results = model.val()
# results = model('../RUSSIAN_PLANAE_DB/original/0266.png')